In [32]:
import pandas as pd
from welleng.error_formula_extractor.enums import Propagation, VectorType
from welleng.error_formula_extractor.models import ErrorTerm, SurveyToolErrorModel
from welleng.error_formula_extractor.formula_utils import function_builder


In [9]:
dfs = pd.read_excel('error-model-example-mwdrev5-1-iscwsa-1 (1).xlsx',sheet_name="Model", usecols="D:W", header=2)

In [10]:
dfs.columns

Index(['No', 'Code', 'Term Description', 'Wt.Fn.', 'Wt.Fn. Source', 'Type',
       'Magnitude', 'Units', 'Prop.', 'P1', 'P2', 'P3', 'Wt.Fn. Comment',
       'Depth Formula', 'Inclination Formula', 'Azimuth Formula',
       'Singularity North Formula', 'Singularity East Formula',
       'Singularity Vert. Formula', 'Convert Magnitudes Degrees to Radians'],
      dtype='object')

In [11]:
desired_cols = ['No', 'Code', 'Units', 'Prop.',
       'Depth Formula', 'Inclination Formula', 'Azimuth Formula',
        'Convert Magnitudes Degrees to Radians']
df = dfs[desired_cols]

In [12]:
desired_cols = {
    "No": "sequence_no",
    'Code': "term_name", 
    'Units': "c_units", 
    'Prop.': "tie_type",
    'Convert Magnitudes Degrees to Radians': "c_value",
    'Depth Formula': 'Depth Formula', 
    'Inclination Formula': 'Inclination Formula', 
    'Azimuth Formula': 'Azimuth Formula'
}
df = dfs[desired_cols.keys()]

In [13]:
df.columns = desired_cols.values()

In [14]:
df

,sequence_no,term_name,c_units,tie_type,c_value,Depth Formula,Inclination Formula,Azimuth Formula
0,1,DRFR,m,R,3.500000e-01,1,0,0
1,2,DSFS,-,S,5.600000e-04,MD,0,0
2,3,DSTG,1/m,G,2.500000e-07,MD * TVD,0,0
3,4,ABXY-TI1S,m/s2,S,4.000000e-03,0,-Cos(Inc) / Gfield,(Tan(Dip) * Cos(Inc) * Sin(AzM)) / Gfield
4,5,ABXY-TI2S,m/s2,S,4.000000e-03,0,0,((Tan((pi / 2) - Inc)) - Tan(Dip) * Cos(AzM)) ...
5,6,ABZ,m/s2,S,4.000000e-03,0,-Sin(Inc) / Gfield,Tan(Dip) * Sin(Inc) * Sin(AzM) / Gfield
6,7,ASXY-TI1S,-,S,5.000000e-04,0,Sin(Inc) * Cos(Inc) / Sqr(2),(-Tan(Dip) * Sin(Inc) * Cos(Inc) * Sin(AzM)) /...
7,8,ASXY-TI2S,-,S,5.000000e-04,0,Sin(Inc) * Cos(Inc) / 2,(-Tan(Dip) * Sin(Inc) * Cos(Inc) * Sin(AzM)) / 2
8,9,ASXY-TI3S,-,S,5.000000e-04,0,0,(Tan(Dip) * Sin(Inc) * Cos(AzM) - Cos(Inc)) / 2
9,10,ASZ,-,S,5.000000e-04,0,-Sin(Inc) * Cos(Inc),Tan(Dip) * Sin(Inc) * Cos(Inc) * Sin(AzM)


In [21]:
var_map = {

            "MD-MDPrev": "dmd",
            "AzT-AzPrev": "dazt",
            "MD": "tmd",
            "TVD": "tvd",
            "Inc": "inc",
            "Gfield": "gtot",
            "Dip": "dip",
            "BField": "mtot",
            "AzT": "azt",
            "AzM": "azm",
        }



In [16]:
formula_cols = ['Depth Formula', 'Inclination Formula', 'Azimuth Formula']

In [27]:
def replace_str(formula_str):
    formula_str = str(formula_str)
    for search, replace in var_map.items():
        formula_str = formula_str.replace(search, replace)

    if formula_str == "0":
        return None
    return formula_str.lower()
replace_str("Max(Abs(AzT-AzPrev), 0.0328084 * (MD-MDPrev) / Sin(Inc))")

'max(abs(dazt), 0.0328084 * (dmd) / sin(inc))'

In [28]:
for col in formula_cols:
    df[col]  = df[col].apply(replace_str)
df

C:\Users\Deep Joshi\AppData\Local\Temp\ipykernel_2052\2430101467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col]  = df[col].apply(replace_str)


,sequence_no,term_name,c_units,tie_type,c_value,Depth Formula,Inclination Formula,Azimuth Formula
0,1,DRFR,m,R,3.500000e-01,1,None,None
1,2,DSFS,-,S,5.600000e-04,tmd,None,None
2,3,DSTG,1/m,G,2.500000e-07,tmd * tvd,None,None
3,4,ABXY-TI1S,m/s2,S,4.000000e-03,None,-cos(inc) / gtot,(tan(dip) * cos(inc) * sin(azm)) / gtot
4,5,ABXY-TI2S,m/s2,S,4.000000e-03,None,None,((tan((pi / 2) - inc)) - tan(dip) * cos(azm)) ...
5,6,ABZ,m/s2,S,4.000000e-03,None,-sin(inc) / gtot,tan(dip) * sin(inc) * sin(azm) / gtot
6,7,ASXY-TI1S,-,S,5.000000e-04,None,sin(inc) * cos(inc) / sqr(2),(-tan(dip) * sin(inc) * cos(inc) * sin(azm)) /...
7,8,ASXY-TI2S,-,S,5.000000e-04,None,sin(inc) * cos(inc) / 2,(-tan(dip) * sin(inc) * cos(inc) * sin(azm)) / 2
8,9,ASXY-TI3S,-,S,5.000000e-04,None,None,(tan(dip) * sin(inc) * cos(azm) - cos(inc)) / 2
9,10,ASZ,-,S,5.000000e-04,None,-sin(inc) * cos(inc),tan(dip) * sin(inc) * cos(inc) * sin(azm)


In [30]:
Survey_Tool = SurveyToolErrorModel(
        survey_tool_id="none",
    survey_tool_name = "A001Mc",
    sequence_no = 0,
    error_terms = [],
    start_depth= 0,
    end_depth = 8000

)

In [52]:
def extract_tie_type(code: dict) -> str:
    if code["tie_type"].lower() == 'r':
        return Propagation.RANDOM
    elif code["tie_type"].lower()  == "s":
        return Propagation.SYSTEMATIC
    elif code["tie_type"].lower()  == "g":
        return Propagation.GLOBAL
    elif code["tie_type"].lower()  == "n":
        return Propagation.NA

    return Propagation.SYSTEMATIC

def create_error_class(row, formula: str, vector: VectorType):
    tie_type = extract_tie_type(row)

    return ErrorTerm(
        sequence_no = row["sequence_no"],
        term_name = row["term_name"],
        formula= formula,
        error_function=(result := function_builder(formula, row["term_name"].lower().replace("-", "_")))[0],
        magnitude=row["c_value"],
        units=row["c_units"],
        tie_type=tie_type,
        vector_type= vector,
        arguments=result[1],
        func_string=result[2]

    )

In [53]:
def convert_error_terms(row):
    print(row)
    error_terms = []

    if row["Depth Formula"]:
        formula = row["Depth Formula"]
        vector = VectorType.DEPTH
        error_terms.append(create_error_class(row, formula, vector))

    if row["Inclination Formula"]:
        formula = row["Inclination Formula"]
        vector = VectorType.INCLINATION
        error_terms.append(create_error_class(row, formula, vector))

    if row["Azimuth Formula"]:
        formula = row["Azimuth Formula"]
        if row["Inclination Formula"]:
            vector = VectorType.LATERAL
        else:
            vector = VectorType.AZIMUTH

        error_terms.append(create_error_class(row, formula, vector))

    return error_terms


In [54]:
error_terms = []
for index, row in df.iterrows():
    new_error_terms = convert_error_terms(row)
    error_terms.extend(new_error_terms)

sequence_no               1
term_name              DRFR
c_units                   m
tie_type                  R
c_value                0.35
Depth Formula             1
Inclination Formula    None
Azimuth Formula        None
Name: 0, dtype: object
sequence_no                  2
term_name                 DSFS
c_units                      -
tie_type                     S
c_value                0.00056
Depth Formula              tmd
Inclination Formula       None
Azimuth Formula           None
Name: 1, dtype: object
sequence_no                    3
term_name                   DSTG
c_units                      1/m
tie_type                       G
c_value                      0.0
Depth Formula          tmd * tvd
Inclination Formula         None
Azimuth Formula             None
Name: 2, dtype: object
sequence_no                                                  4
term_name                                            ABXY-TI1S
c_units                                                   m/s2
tie_ty

In [56]:
len(error_terms)

42